#Cipres Quick Start with python_cipres
This is a version of the [Cipres Quick Start Tutorial](https://www.phylo.org/restusers/docs/qs.html) that uses the python_cipres client instead of curl.  

First we install the python_cipres package if it isn't already installed:

In [ ]:
# Support for both Python 2 and 3
from __future__ import print_function

In [ ]:
!pip install python_cipres

In [ ]:
import python_cipres.client as CipresClient
import requests
import os

If you registered an application of your own, replace appname and appid with yours.  Otherwise you can run as the "demo" application.

You can create an application from: https://www.phylo.org/restusers/developer

In [ ]:
url = "https://cipresrest.sdsc.edu/cipresrest/v1/"
appname = "demo"
appid = "mydemo-8126CB900A964FA1AD14174512F9403C"

**CHANGE THIS - Fill in your own username and password:**

In [ ]:
username = ADD_YOUR_USERNAME_HERE
password = ADD_YOUR_PASSWORD_HERE

We create a CipresClient to communicate with the CIPRES REST API:

In [ ]:
client = CipresClient.Client(appname, appid, username, password, url)

Here we get the demo's sample phylip file to use as input to raxml.   Python_cipres expects the input data to be in a file on disk.

In [ ]:
inputDataUrl = "https://www.phylo.org/restusers/docs/raxml_inputphy.txt"
inputData = requests.get(inputDataUrl).text;
inputFilename = './raxml_inputphy.txt'
fd = open(inputFilename, mode='w')
fd.write(inputData)
fd.close()

Now we submit the job to Cipres and ask for email notification when the job is finished.

In [ ]:
job = client.submitJob(
            {'tool' : 'RAXMLHPC8_REST_XSEDE'},
            {'input.infile_': inputFilename},
            {'metadata.statusEmail': 'true'});

Display status information about the job.

In [ ]:
job.show(messages="true")

OK, the job was submitted. You can update and display the job's status as often as you like by re-running the following cell:

In [ ]:
job.update()
job.show(messages="true")

This is how you would programmatically wait for the job to finish.  **Wait for the "Job ... is finished" output to appear before trying to do anything else:**

In [ ]:
job.waitForCompletion()
print("Job %s finished.  isError() returns %s" % (job.jobHandle, job.isError()))

Now we're going to download the result files:

In [ ]:
downloadDir = "./myResults"
try:
    os.mkdir(downloadDir)
except:
    pass
job.downloadResults(downloadDir)

In [ ]:
print(os.listdir(downloadDir))

We can look at one of the results, [RAxML_bestTree.results](./myResults/RAxML_bestTree.result) as a raw newick text file.  Or, if biopython is installed, we could use it to view the tree, like this:

In [ ]:
from Bio import Phylo
tree = Phylo.read("./myResults/RAxML_bestTree.result", "newick")
Phylo.draw_ascii(tree)

Here's how to list your jobs:

In [ ]:
jobs = client.listJobs()
for j in jobs:
    j.show()

Once you've downloaded a job's results you should delete the job to conserve space on CIPRES.  If you delete a job that is still running or is queued to run, it will be cancelled.

In [ ]:
job.delete()

You can go back and re-run the cell that lists your jobs and verify that the one you deleted is no longer in the list.